In [45]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup

r = requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

In [46]:
raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
card_list[-1]

'Zhalfirin Void'

In [47]:
def get_card_data(card_name):
    r = requests.get('https://api.scryfall.com/cards/search/?q=' + card_name)
    return(r.json()) # we _should_ always get the card we need since titles are exact
pool = mp.Pool()
card_data = pool.map(get_card_data, card_list)
pool.close()

In [56]:
card_df = pd.DataFrame(list(map(lambda x: x['data'][0], card_data)))
card_df = card_df[['name','type_line','cmc','mana_cost','oracle_text','power','toughness','usd','uri']]
card_df.head()

,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd,uri
0,"Muldrotha, the Gravetide",Legendary Creature — Elemental Avatar,6.0,{3}{U}{B}{G},"During each of your turns, you may play up to ...",6,6,5.99,https://api.scryfall.com/cards/dom/199
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or a...,NaN,NaN,5.17,https://api.scryfall.com/cards/mm3/146
2,Acidic Slime,Creature — Ooze,5.0,{3}{G}{G},Deathtouch (Any amount of damage this deals to...,2,2,0.25,https://api.scryfall.com/cards/cma/90
3,Alchemist's Refuge,Land,0.0,,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: ...",NaN,NaN,1.99,https://api.scryfall.com/cards/avr/225
4,Arcane Lighthouse,Land,0.0,,"{T}: Add {C} to your mana pool.\n{1}, {T}: Unt...",NaN,NaN,2.79,https://api.scryfall.com/cards/c14/59
